In [6]:
"""
This module is designed to generate AUCs for testing the Half Site Hypothesis.

NJM
6/13/18


Modified by PSR
"""

from KdtoEv2 import psfem
from KdtoEv2 import get_psfem
from KdtoEv2 import make_comparison_frame,add_psfem_info, getScores,get_tpr_fpr_lists, plotROCCurve, getSingleScore, getDoubleScore
from analyze_sig_hits_v2 import pssm, add_pwm_info,make_comparison_frame, get_pssm, get_max_pwm_score, one_weak_one_strong
from math import log,e
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
from sklearn import metrics
import os


def removeCannon(df, cutoff = 13.01, tf = 'Gal4'):
	"""This function removes canonical site based on a PWM cutoff"""
	df["Score"] = df["Sequence"].apply(lambda x: get_max_pwm_score(x,tf))
	newdf = df[df["Score"]<cutoff]
	return newdf

def foo(lowerCutoff = .00001, upperCutoff = .1, tf = 'Gal4', testSeq = ""):
	""" This function uses the total number of half sites as a model to differentiate between bound and unbound promoters"""
	cf = make_comparison_frame('sig_prom_Gal4_FL_HarbIG_Final.txt','sig_prom_Gal4_DBD_HarbIG_Final.txt','FL','DBD')
	cf = removeCannon(cf)
	positives = cf[cf["FL pvalue"]<lowerCutoff]
	negatives = cf[cf["FL pvalue"]>upperCutoff]
	tpr,fpr = get_tpr_fpr_lists(positives,negatives,lambda x: getSpacingMax400AndUp(x, testSeq),False)
	auc = metrics.auc(fpr,tpr)
	return auc

def getSpacingMax400AndUp(inSeq, testSeq):
	"""This function returns the max number of halfsites for a promoter sequence"""
	seq = Seq(str(inSeq), IUPAC.unambiguous_dna)
	totals = []
	for x in range(0,len(seq)-399):
		inputSeq = seq[x:x+400]
		totals.append(FindConsensusMotif(testSeq,inputSeq))
	if totals:
		return max(totals)
	else:
		return 0



In [37]:
import Bio
from Bio.Seq import Seq
from Bio.Alphabet.IUPAC import unambiguous_dna, ambiguous_dna
from Bio import SeqUtils

def FindConsensusMotif(testseq,seq):
    
    ''' This function takes as input a "consensus motif", which is essentially a consensus sequence,
    but with Ns,Rs,Ys,Ks, and a sequence to scan. It will scan the sequence and return 
    a list of tuples that give the start and stop coordinates of all instances of the 
    consensus motif, on both strands of course.
    
    The consensusMotif variable can be passed to this function a string or a Seq object 
    The input_sequence variable can also be baseed as a string or Seq object'''

    input_sequence = Seq(str(seq),ambiguous_dna)
    consensusMotif = Seq(str(testseq),ambiguous_dna)
    
    #find all instances of consensus motif in forward strand of the sequence
    start_positions_forward = SeqUtils.nt_search(str(input_sequence), str(consensusMotif))[1:]
    
    #find all instances of consensus motif in the reverse strand of the sequence
    
    #first, take the reverse complement of the consensus motif
    rc_consensusMotif = consensusMotif.reverse_complement()
    
    if rc_consensusMotif == consensusMotif: #if consensusMotif is palindromic, don't search reverse strand
        end_positions_reverse = []
    else:
        end_positions_reverse = SeqUtils.nt_search(str(input_sequence), str(rc_consensusMotif))[1:]
    
    #output list of lists giving start and end positions of all motif 
    #instances.  For instance on the forward strand, start < end.  
    #For reverse strand, start > end.  The indices are zero indexed.
    output_list = []
    for startpos in start_positions_forward:
        position_list = [startpos,startpos+len(consensusMotif)-1]
        output_list.append(position_list)
    
    for endpos in end_positions_reverse:
        position_list = [endpos+len(consensusMotif)-1,endpos]
        output_list.append(position_list)
 
    
    return len(output_list)

In [38]:
alt_map = {'ins':'0'}
complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'} 

def reverse_complement(seq):    
    for k,v in alt_map.iteritems():
        seq = seq.replace(k,v)
    bases = list(seq) 
    bases = reversed([complement.get(base,base) for base in bases])
    bases = ''.join(bases)
    for k,v in alt_map.iteritems():
        bases = bases.replace(v,k)
    return bases


In [40]:
#generates every possible combination of 3mer sequences 

letters = ["A","T","C","G"]
arr = []
testArray = []
res = []
for x in itertools.product(letters,repeat = 3):
	arr.append(x)     

#print(len(arr))
for x in arr:
	iteration = ""
	for y in x:
		iteration = iteration + str(y)
	testArray.append(iteration)


for i in testArray:
    if i not in res and reverse_complement(i) not in res:
        res.append(i)

#len(res)
#res = ['CGG', 'CCG']


#AUC is calculated for each 3mer sequence     
finalFrame = pd.DataFrame(columns = ["Seq", "AUC"])
i = 0
for x in res:
	i = i + 1
 	auc = foo(testSeq = str(x))
	#print(auc)
	itterFrame = pd.DataFrame([[str(x),auc]], columns = ["Seq", "AUC"])
	finalFrame = finalFrame.append(itterFrame)

finalFrame.to_csv("CGG_rich_test_result_FINAL.txt", sep = '\t')


